In [1]:
import pandas as pd
import numpy as np
import allel
import moments.LD
import scipy.interpolate
import msprime as ms

### Preprocessing of 1000G VCF files
1. Apply strict mask
    - Some sites are not reliably mapped in hg38 and need to be excluded.
    - These regions include: repeat regions, pseudogenes, centromeric, and telomeric regions.
2. Select only biallelic SNP sites
    - A VCF can contain mult-allelic SNPs and structure variants (indels, tandem repeats, etc)
    - The biallelic SNP sites are the easiest to work with, and most studies focus on them. 



This process can take quite a long time, I saved the processed VCFs under ~/projects/ctb-sgravel/data/30x1000G_biallelic_strict_masked <br>
The example preprocessing script is provided below and also in the directory's README.md.

In [ ]:
%%sh
### in bash, preprocess using bcftools
### replace variable path
chr22_vcf_path='/home/alouette/projects/ctb-sgravel/data/1000G/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr22.recalibrated_variants.vcf.gz'
strict_mask_1000G='/home/alouette/projects/ctb-sgravel/data/1000G/masks_and_annotations/20160622.allChr.mask.bed'
IntegerT=6
module load bcftools
bcftools --version | head -1
### -Oz output compressed VCF format
### -m2 -M2 -v to only view biallellic SNPs
### -R subset VCF from region bed file
### --threads parallelization currently only works for compression
bcftools view -i 'FILTER=="PASS"' -Oz -m2 -M2 -v snps -R ${strict_mask_1000G} ${chr22_vcf_path} --threads ${IntegerT} -o "chr22.strict_masked.vcf.gz"

bcftools manual: http://samtools.github.io/bcftools/bcftools.html#expressions

### Optional: covert the VCF to zarr format for faster retrieval in scikit-allele
In sgkit, there might be better approaches. <br>
I also completed this step and stored the output in ~/projects/ctb-sgravel/data/30x1000G_biallelic_strict_masked/zarrFormat

### Population information retrieval
1000G stored its population information in a txt file: ~/projects/ctb-sgravel/data/1000G/population_info/20130606_g1k_3202_samples_ped_population.txt <br>
This file includes family ID, sample ID, parent ID, sex, population, superpopulation. <br>
There are some family trio in 1000G data. I currently exclude the individuals information if any of their parents are also in 1000G. <br>


### Formatting the recombination map for pipeline
Several recombination maps exist for the human population. Each is computed using specific populations and with specific techniques.<br>
Two major formats exist: HapMap format and PLINK format. <br>
The main difference is how recombination rate is stored:
- HapMap: Chromosome, Position(bp), Rate(cM/Mb)
- PLINK: Chromosome, Cumulative Map(cM), Position

Recombination rate starts at 0 at the first measured SNP position, instead of starting at the first chromosome position. <br>
The current ~/projects/ctb-sgravel/data/genetic_maps contains many maps in Hg37 format. <br>
I used the HapMapII map in ~/projects/ctb-sgravel/data/genetic_maps/HapMapII_GRCh38. But transformed it into cumulative Map.

### Minimal example pipeline: EDAR region in chr2:107,500,000-110,000,000
EDAR is an identified sweep in the EAS population.

In [ ]:
%%sh
chr2_vcf_path='/home/alouette/projects/ctb-sgravel/data/1000G/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr2.recalibrated_variants.vcf.gz'
strict_mask_1000G='/home/alouette/projects/ctb-sgravel/data/1000G/masks_and_annotations/20160622.allChr.mask.bed'
IntegerT=6
module load bcftools
### takes 6 min
bcftools view -i 'FILTER=="PASS"' -Oz -m2 -M2 -v snps -r chr2:107500000-110000000 ${chr2_vcf_path} --threads ${IntegerT} -o "chr2.EDAR_region.vcf.gz"
bcftools index --threads ${IntegerT} "chr2.EDAR_region.vcf.gz"
### takes 3 min
bcftools view -Oz -R ${strict_mask_1000G} "chr2.EDAR_region.vcf.gz" --threads ${IntegerT} -o "chr2.EDAR_region.strict_mask.vcf.gz"

In [2]:
def read_vcf(vcf_path:str) -> dict: 
    """
    Read in the VCF using scikit-allel, the genotype information is stored in a dict
    Optimization: could limit the fields read in, can also specify detailed format like float32
    """
    callset = allel.read_vcf(vcf_path)
    return callset

In [3]:
vcf_path = "chr2.EDAR_region.strict_mask.vcf.gz"
EDAR_callset = read_vcf(vcf_path)

In [4]:
EDAR_callset

{'samples': array(['HG00096', 'HG00097', 'HG00099', ..., 'NA21142', 'NA21143',
        'NA21144'], dtype=object),
 'calldata/GT': array([[[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        ...,
 
        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]], dtype=int8),
 'variants/ALT': array([['G', '', ''],
        ['T', '', ''],
        ['T', '', ''],
        ...,
        ['A', '', ''],
        [

In [15]:
len(EDAR_callset['variants/POS'])

58860

In [46]:
type(EDAR_callset['variants/POS'])

numpy.ndarray

In [14]:
pos_array = EDAR_callset['variants/POS']
#snp_array = EDAR_callset['calldata/GT']

In [16]:
np.min(pos_array)

107500027

In [17]:
np.max(pos_array)

109743205

In [5]:
def _read_rec_map(rec_map_path: str) -> pd.DataFrame:
    """
    Read the recombination map, normalize between different formats.
    Currently set to read from HapMap format, with a one row header
    Covert the Rate(cM/MB) to cumulative Rate (cM)
    Return a pd.DataFrame
    """
    rec_map_df = pd.read_csv(rec_map_path, sep = "\t", names = ["Chr", "bp", "Rate_cM"], skiprows = 1)
    rec_map_df["diff_bp"] = rec_map_df.bp.diff()[1:].reset_index(drop = True)
    rec_map_df.diff_bp = rec_map_df.diff_bp.astype("Int32")
    rec_map_df["cM"] = rec_map_df.diff_bp * rec_map_df.Rate_cM / 1e6
    rec_map_df["cum_cM"] = rec_map_df.cM.cumsum()
    return rec_map_df

In [6]:
def _msprime_read_HapMap(rec_map_path: str) -> pd.DataFrame:
    """
    Require msprime >= 1.0.0
    Read the recombination map using msprime functionality.
    Can uses the functionality: get_cumulative_mass(), find_index()
    Column 1-2, positions; left: inclusive, right: exclusive
    """
    ms_RateMap = ms.RateMap.read_hapmap(rec_map_path, position_col=1, rate_col=2)
    return ms_RateMap

In [74]:
def _bp_to_cM(ms_RateMap: ms.RateMap) -> np.ndarray:
    """
    Return the function transforming bp to cM.
    """
    bp_to_cM_F = scipy.interpolate.interp1d(ms_RateMap.left, np.nancumsum(ms_RateMap.mass)*100, fill_value='extrapolate' )
    return bp_to_cM_F

In [75]:
def _cM_to_bp(ms_RateMap: ms.RateMap) -> np.ndarray:
    """
    Return the function transforming cM to bp.
    """
    cM_to_bp_F = scipy.interpolate.interp1d(np.nancumsum(ms_RateMap.mass)*100, ms_RateMap.left, fill_value='extrapolate' )
    return cM_to_bp_F

In [33]:
def _get_bp_from_cum_cM(cM_list: list, ms_RateMap: ms.RateMap) -> np.ndarray:
    """
    Following msprime RateMap get_cumulative_mass functionality for the bp to cumulative cM convertion.
    """
    M_array = np.array(cM_list)/100 ### convert to Morgan
    if np.any(M_array < 0) or np.any(M_array > ms_RateMap.total_mass):
        raise ValueError(f"Cannot have cumulative cM < 0 or > {ms_RateMap.total_mass * 100}")
    pos_array = np.interp(M_array, np.nancumsum(ms_RateMap.mass), ms_RateMap.left)
    pos_array = pos_array.astype(int)
    return pos_array

In [8]:
def _get_cum_cM_from_bp(pos_list: list, ms_RateMap: ms.RateMap) -> np.ndarray:
    """
    Using msprime RateMap functionality
    """
    return ms_RateMap.get_cumulative_mass(pos_list)*100

In [76]:
cM_to_bp_F = _cM_to_bp(rec_map_df)

In [125]:
_get_cum_cM_from_bp([133762002, 48232], rec_map_df)

array([1.78351170e+02, 2.66429381e-03])

In [115]:
_get_bp_from_cum_cM(178.350202, rec_map_df)

133634496

In [58]:
rec_map_path = "/home/alouette/projects/ctb-sgravel/data/genetic_maps/HapMapII_GRCh38/genetic_map_Hg38_chr2.txt"
rec_map_df2 = _read_rec_map(rec_map_path)
rec_map_df2

,Chr,bp,Rate_cM,diff_bp,cM,cum_cM
0,chr2,12994,0.339408,2497,0.000848,0.000848
1,chr2,15491,0.336057,181,0.000061,0.000908
2,chr2,15672,0.308424,31,0.00001,0.000918
3,chr2,15703,0.273441,408,0.000112,0.001029
4,chr2,16111,0.232868,826,0.000192,0.001222
...,...,...,...,...,...,...
286388,chr2,242106373,0.122849,236,0.000029,267.431831
286389,chr2,242106609,0.133203,3571,0.000476,267.432306
286390,chr2,242110180,0.136364,9179,0.001252,267.433558
286391,chr2,242119359,0.174241,29487,0.005138,267.438696


In [100]:
rec_map_df

left,right,mid,span,rate
0,26829,13414.5,26829,nan
26829,48232,37530.5,21403,1.2e-09
48232,48486,48359,254,1.6e-09
48486,50009,49247.5,1523,1.6e-09
50009,52147,51078,2138,1.6e-09
52147,52541,52344,394,1.6e-09
52541,64718,58629.5,12177,1.6e-09
64718,66015,65366.5,1297,1.6e-09
66015,67284,66649.5,1269,1.6e-09
67284,67994,67639,710,1.7e-09


In [33]:
rec_map_msprime.get_cumulative_mass(133766368)

1.783658341121365

In [60]:
bp_to_cM_F = rec_map_interpolate(rec_map_df)

In [61]:
bp_to_centimorgan(EDAR_callset['variants/POS'], rec_map_df)

array([0.00525299, 0.0052575 , 0.005267  , ..., 2.73506194, 2.73505867,
       2.73505847])

### sgkit window function inspiration 

sgkit has 3 windowing functions: by variant, by position, and by genome.

In [ ]:
### sgkit window master function functions
    n_variants = ds.sizes["variants"]
    n_contigs = num_contigs(ds)
    contig_ids = np.arange(n_contigs)
    variant_contig = ds["variant_contig"]
    contig_starts = np.searchsorted(variant_contig.values, contig_ids)
    contig_bounds = np.append(contig_starts, [n_variants], axis=0)  # type: ignore[no-untyped-call]

    contig_window_contigs = []
    contig_window_starts = []
    contig_window_stops = []
    for i, contig in enumerate(get_contigs(ds)):
        starts, stops = windowing_fn(
            contig, contig_bounds[i], contig_bounds[i + 1], *args, **kwargs
        )
        contig_window_starts.append(starts)
        contig_window_stops.append(stops)
        contig_window_contigs.append(np.full_like(starts, i))

    window_contigs = np.concatenate(contig_window_contigs)  # type: ignore[no-untyped-call]
    window_starts = np.concatenate(contig_window_starts)  # type: ignore[no-untyped-call]
    window_stops = np.concatenate(contig_window_stops)  # type: ignore[no-untyped-call]

### sgkit window function, window_by_position
if step is not None and window_start_position is not None:
        raise ValueError("Only one of step or window_start_position may be specified")
    step = step or size
    positions = ds[variant_position].values
    window_start_positions = (
        ds[window_start_position].values if window_start_position is not None else None
    )
    return _window_per_contig(
        ds,
        variant_contig,
        merge,
        _get_windows_by_position,
        size,
        step,
        offset,
        positions,
        window_start_positions,
    )

In [70]:
def _windowing(snp_pos: list, window_pos_list:list) -> list:
    """
    Master window function, based on the input bp, give back the subset windows in a list
    This function is the core function and used for other detailed window_by_* functions.
    :param snp_array: The SNP array read by scikit-allel format.
    :param window_index_array: The list of tuple (start, end) index of each windows of the SNP array.
    Return a list of slices
    """
    if np.any(window_pos_list) < 0 or np.max(window_pos_list) > np.max(snp_pos) or np.min(window_pos_list) > np.min(snp_pos):
        raise ValueError(f"Window positions out of bounds")
    if not np.all(np.diff(snp_pos) > 0):
        raise ValueError(f"SNP positions are not in asending order")
    if not np.all(np.diff(window_pos_list) > 0):
        raise ValueError(f"Window positions are not in asending order")
    window_list = []
    for start, end in zip(window_pos_list[:-1], window_pos_list[1:]):
        start_index =np.searchsorted(snp_pos, start, side='right')
        end_index = np.searchsorted(snp_pos, end, side='right')
        if start_index == end_index:
            ### recombination hotspot can result in no SNPs in the slice
            continue
        window_list.append(slice(start_index, end_index))
    ### next step requires loading the data, do it when in need
    return window_list

In [69]:
index_snp_pos

0,1,2,3,4,...,58855,58856,58857,58858,58859
107500027,107500073,107500170,107500252,107500306,...,109736633,109736825,109736849,109742841,109743205


In [62]:
index_snp_pos = allel.SortedIndex(pos_array)
index_snp_pos.locate_range(107811389)

slice(9827, 58860, None)

In [71]:
def window_by_reombination(pos_array: list, rec_map: ms.RateMap, rec_start: float, rec_end: float, rec_step = 0.04) -> list:
    """
    Require msprime >= 1.0.0 to use RateMap
    Call this function to get the window list separated by rec_distance
    How to handle the last position if rec subsetting is not possible?
    """
    ### end has to be larger than start
    if rec_start >= rec_end:
        raise ValueError(f"Input recombination range is invalid")
    ### if given rec distance is smaller or equal to the the step 
    ### return the original array without subsetting
    if rec_end - rec_start <= rec_step:
        rec_list = [rec_start, rec_end]                    
    else:
        rec_list = np.arange(start = rec_start, stop = rec_end, step = rec_step)
        ### check the last position
        ### floating point issue, the last position can be larger than end
        if rec_list[-1] > rec_end:
            rec_list[-1] = rec_end
    print(rec_list)
    ### get bp positions
    bp_array = _get_bp_from_cum_cM(rec_list, rec_map)
    print(bp_array)
    ### call _windowing function
    window_list = _windowing(pos_array, bp_array)
    return window_list
    
    

In [72]:
pos_array = EDAR_callset['variants/POS']
rec_map_path = "/home/alouette/projects/ctb-sgravel/data/genetic_maps/HapMapII_GRCh38/genetic_map_Hg38_chr2.txt"
ms_RateMap = _msprime_read_HapMap(rec_map_path)
rec_range = _get_cum_cM_from_bp([pos_array.min(), pos_array.max()], ms_RateMap)
window_list = window_by_reombination(pos_array, ms_RateMap, rec_range[0], rec_range[1], 0.04)

[125.53773142 125.57773142 125.61773142 125.65773142 125.69773142
 125.73773142 125.77773142 125.81773142 125.85773142 125.89773142
 125.93773142 125.97773142 126.01773142 126.05773142 126.09773142
 126.13773142 126.17773142 126.21773142 126.25773142 126.29773142
 126.33773142 126.37773142 126.41773142 126.45773142 126.49773142
 126.53773142 126.57773142 126.61773142 126.65773142 126.69773142
 126.73773142 126.77773142 126.81773142 126.85773142 126.89773142
 126.93773142 126.97773142 127.01773142 127.05773142 127.09773142
 127.13773142 127.17773142 127.21773142 127.25773142 127.29773142
 127.33773142 127.37773142 127.41773142 127.45773142 127.49773142
 127.53773142 127.57773142 127.61773142 127.65773142 127.69773142
 127.73773142]
[107499693 107538848 107686795 107746830 107766953 107811513 107811544
 107815310 107994804 108008430 108026986 108101392 108104822 108163768
 108278373 108311081 108311184 108311286 108318695 108330509 108331644
 108473257 108712912 108923464 108923555 10892

In [73]:
window_list

[slice(0, 1276, None),
 slice(1276, 5953, None),
 slice(5953, 7781, None),
 slice(7781, 8432, None),
 slice(8432, 9831, None),
 slice(9831, 9949, None),
 slice(9949, 12366, None),
 slice(12366, 12755, None),
 slice(12755, 13300, None),
 slice(13300, 15494, None),
 slice(15494, 15599, None),
 slice(15599, 17232, None),
 slice(17232, 20372, None),
 slice(20372, 21297, None),
 slice(21297, 21492, None),
 slice(21492, 21870, None),
 slice(21870, 21917, None),
 slice(21917, 25438, None),
 slice(25438, 29554, None),
 slice(29554, 33818, None),
 slice(33818, 33822, None),
 slice(33822, 33929, None),
 slice(33929, 35139, None),
 slice(35139, 35847, None),
 slice(35847, 36446, None),
 slice(36446, 36461, None),
 slice(36461, 36834, None),
 slice(36834, 37207, None),
 slice(37207, 37791, None),
 slice(37791, 38572, None),
 slice(38572, 42086, None),
 slice(42086, 45357, None),
 slice(45357, 46801, None),
 slice(46801, 48398, None),
 slice(48398, 48417, None),
 slice(48417, 50315, None),
 slice(5

In [20]:
pos_array.min()

107500027

In [21]:
pos_array.max()

109743205

In [52]:
np.interp([1.2572175346, 1.2576300247], np.nancumsum(ms_RateMap.mass), ms_RateMap.left).astype(int)

array([107811389, 107811533])

In [53]:
_get_cum_cM_from_bp([107811389, 107811533], ms_RateMap)

array([125.72130564, 125.72255928])

In [59]:
rec_map_df2[rec_map_df2.bp >= 107811389]

,Chr,bp,Rate_cM,diff_bp,cM,cum_cM
131177,chr2,107811503,40.290651,58,0.002337,125.723687
131178,chr2,107811561,47.127122,1590,0.074932,125.79862
131179,chr2,107813151,47.127122,31,0.001461,125.80008
131180,chr2,107813182,47.127122,50,0.002356,125.802437
131181,chr2,107813232,47.127122,83,0.003912,125.806348
...,...,...,...,...,...,...
286388,chr2,242106373,0.122849,236,0.000029,267.431831
286389,chr2,242106609,0.133203,3571,0.000476,267.432306
286390,chr2,242110180,0.136364,9179,0.001252,267.433558
286391,chr2,242119359,0.174241,29487,0.005138,267.438696


In [ ]:
def window_by_position(rec_start: float, rec_end: float, rec_to_pos_func: scipy.interpolate, rec_step = 0.04: float) -> list, list:
    """
    Call this function just to get list of position in corresponding windows
    Return the start position and end position in two lists
    """
    ### end has to be larger than start
    assert rec_start < rec_end
    if rec_end - rec_start <= rec_step:
        return [rec_to_pos_func(rec_start)], [rec_to_pos_func(rec_end)]
    else:
        window_counts = (rec_end - rec_start)//rec_step
        
    ### call parsing functions
    
    

In [ ]:
def window_by_counts(rec_start: float, rec_end: float, rec_to_pos_func: scipy.interpolate, rec_step = 0.04: float) -> list, list:
    """
    Call this function just to get list of position in corresponding windows
    Return the start position and end position in two lists
    """
    ### end has to be larger than start
    assert rec_start < rec_end
    if rec_end - rec_start <= rec_step:
        return [rec_to_pos_func(rec_start)], [rec_to_pos_func(rec_end)]
    else:
        window_counts = (rec_end - rec_start)//rec_step
        
    ### call parsing functions
    
    

In [ ]:
def window_Dz(SNP_window):
    """
    
    """
    

In [ ]:
def subset_Dz_computation(snp_window, max_snp_per_subset = 5000):
    """
    Memory requirement is large when there are too many SNPs in a single calculation.
    Ideally the total SNP should be less than 5,000
    Subset the Dz caclulation into chunks and return the combined computed values.
    """
    